### Import Libraries

In [64]:
import psycopg2
import psycopg2.extras
import configparser as configparser
import pandas as pd
import numpy as np
from string import Template

## Parsing INI File

### Funtion: To parse INI file

In [65]:
def parse_ini(section: str) -> dict:
    """
    This function parses ini file for configuration details
    :param section: section to read from ini
    :return: Dictionary of config details
    """
    config = dict()
    parser = configparser.ConfigParser()
    parser.read("imdb_database.ini")
    if parser.has_section(section):
        config_items = parser.items(section)
        for item in config_items:
            config[item[0]] = item[1]
    return config

In [66]:
sql_config = parse_ini("postgresql")
sql_config

{'host': 'localhost',
 'database': 'imdb',
 'user': 'postgres',
 'password': '1997',
 'port': '5432'}

### Function: To connect to IMDB database and run query to fetch from IMDB SQL Database

In [67]:
def fetch_sql(query: str):
    with psycopg2.connect(**sql_config) as conn:
        conn.autocommit = True
        cursor = conn.cursor()
        try:
            cursor.execute(query)
            results = cursor.fetchall()
            return results
        except (Exception, psycopg2.DatabaseError) as error:
            print("SQL Exception:" + str(error))
        finally:
            cursor.close()

## Itemset Mining on IMDB for Actors who have appeared together

### Template query for Itemset Mining on IMDB DB

In [68]:
query_0 = Template("DROP TABLE IF EXISTS L${level};")
query_1 = Template(("\nCREATE TABLE L${level} AS"
           "\nSELECT"))
query_2 = Template(" p${Lk_current}.actor AS actor${Lk_current},")
query_3 = (", COUNT(p1.title) AS count\n"
           "FROM Popular_Movie_Actors p1")
query_4 = Template(("\nINNER JOIN Popular_Movie_Actors p${Lk_current}"
           "\nON p${Lk_previous}.title = p${Lk_current}.title"
           "\nAND p${Lk_previous}.actor < p${Lk_current}.actor"))
query_5 = "\nGROUP BY"
query_6 = Template(" p${Lk_current}.actor,")
query_7 = Template(("\nHAVING COUNT(p1.title) >= ${minimum_support};"
           "\nSELECT COUNT(*) FROM L${level};"))
query_parts = [query_0, query_1, query_2, query_3, query_4, query_5, query_6, query_7]

### Algorithm to mine actor frequent itemsets from IMDB database

In [69]:
frequent_itemsets_count = []
minimum_support = 5
level = 1
count = np.inf
while count > 0:
    query = ""
    for index in range(len(query_parts)):
        if index in [0, 1, 7, 8]:
            query += str(query_parts[index].safe_substitute(level=level, minimum_support=minimum_support))
        elif index in [2, 6]:
            for Lk_current in range(1, level + 1):
                query += str(query_parts[index].substitute(Lk_current=Lk_current))
            query = query.strip(',')
        elif index == 4:
            if level > 1:
                for Lk_current in range(2, level + 1):
                    query += str(query_parts[index].substitute(Lk_current=Lk_current, Lk_previous=(Lk_current - 1)))
        else:
            query += str(query_parts[index])
    
    count = fetch_sql(query)[0][0]
    if count == 0:
        query = str(query_parts[0].substitute(level=level))
        fetch_sql(query)
        break
    
    frequent_itemsets_count.append(count)
    level += 1

frequent_itemsets_count

SQL Exception:no results to fetch


[33443, 5591, 710, 159, 34, 5]

#### Count of frequent itemsets at each level

In [75]:
for index in range(len(frequent_itemsets_count)):
    print("Level " + str(index + 1) + ": " + str(frequent_itemsets_count[index]))

Level 1: 33443
Level 2: 5591
Level 3: 710
Level 4: 159
Level 5: 34
Level 6: 5


### Querying Last Level lattice to get results with actors' names

In [70]:
query_0 = Template("SELECT")
query_1 = Template(" m${current}.name AS name${current},")
query_2 = Template((", count\n"
           "FROM L${level} l"))
query_3 = Template(("\nINNER JOIN member m${current}"
           "\nON l.actor${current} = m${current}.id"))
query_parts = [query_0, query_1, query_2, query_3]

#### Generated Query

In [71]:
last_level = len(frequent_itemsets_count)
query = ""
for index in range(len(query_parts)):
    if index in [1, 3]:
        for Lk_current in range(1, last_level + 1):
            query += str(query_parts[index].substitute(current=Lk_current))
        query = query.strip(',')
    else:
        query += str(query_parts[index].safe_substitute(level=last_level))
print(query)

SELECT m1.name AS name1, m2.name AS name2, m3.name AS name3, m4.name AS name4, m5.name AS name5, m6.name AS name6, count
FROM L6 l
INNER JOIN member m1
ON l.actor1 = m1.id
INNER JOIN member m2
ON l.actor2 = m2.id
INNER JOIN member m3
ON l.actor3 = m3.id
INNER JOIN member m4
ON l.actor4 = m4.id
INNER JOIN member m5
ON l.actor5 = m5.id
INNER JOIN member m6
ON l.actor6 = m6.id


#### Fetching results

In [72]:
last_level_frequent_itemset = pd.DataFrame(fetch_sql(query))
columns = []
for index in range(1, last_level_frequent_itemset.shape[1]):
    columns.append("actor" + str(index))
columns.append("count")
last_level_frequent_itemset.columns = columns

#### The names of the actors in each frequent itemset of last level

In [74]:
last_level_frequent_itemset

,actor1,actor2,actor3,actor4,actor5,actor6,count
0,Robert Axelrod,G. Larry Butler,David Gerrold,Donald F. Glut,Marieve Herington,Kyle Rea,7
1,Robert Axelrod,G. Larry Butler,David Gerrold,Donald F. Glut,Marieve Herington,Bradford Hill,6
2,Robert Axelrod,G. Larry Butler,David Gerrold,Donald F. Glut,Marieve Herington,Jason Barker,6
3,G. Larry Butler,David Gerrold,Donald F. Glut,Marieve Herington,Bradford Hill,Jason Barker,9
4,Vikram Barn,Simon Minter,Josh Bradley,Ethan Payne,Tobi Brown,Harry Lewis,5
